In [1]:
# libraires for images and data
from PIL import Image
import glob,os
from bs4 import BeautifulSoup
import cv2
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# libraries for deep learning
import torch
import torchvision
from torchvision import transforms, datasets, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
                                                                                                                                                                           
import logging
from Pegasus.api import *

from utils.wf import split_data_filenames, create_ann_list,create_augmented_filelist

/home/scitech/shared-data/mask-detection-workflow/img.png


In [2]:
# When you restart the docker make sure to reinstall these:
#! sudo python3 -m pip install --upgrade pip
#! sudo python3 -m pip install --upgrade Pillow numpy torchvision matplotlib
#! sudo python3 -m pip install pandas torch bs4

#! sudo pip3 install opencv-python 
#! sudo pip3 install --upgrade setuptools
#! sudo pip3 install opencv-python 
#! sudo pip3 install optuna==2.0.0 
#! sudo pip3 install matplotlib 
#! sudo pip3 install torch
#! sudo pip3 install scikit-image
#! sudo pip3 install torchvision 
#! sudo pip3 install pytorchtools
#! sudo pip3 install joblib

In [3]:
# DATA AQUSITION
imagesList = glob.glob('Images2/*.png')
annotationList = glob.glob('annotations2/*.xml')

#DATA SPLIT
train_filenames,val_filenames,test_filenames, files_split_dict = split_data_filenames(imagesList)

train_imgs, train_ann = create_ann_list(train_filenames)
val_imgs, val_ann     = create_ann_list(val_filenames)
test_imgs, test_ann   = create_ann_list(test_filenames)

In [4]:
rc = ReplicaCatalog()

inputFiles = []
for img in imagesList:
    fileName = img.split("/")[1]
    img_file = File(fileName)
    inputFiles.append(img_file)
    rc.add_replica("local", img_file,  os.path.join(os.getcwd(),str(img)))
    
annFiles = []
for ann in annotationList:
    fileName = ann.split("/")[1]
    ann_file = File(fileName)
    annFiles.append(ann_file)
    rc.add_replica("local", ann_file,  os.path.join(os.getcwd(),str(ann)))
rc.write()

In [5]:
# Add Transformations

augment_imgs = Transformation(
                "augment_images",
                site="local",
                pfn=  os.path.join(os.getcwd(),"bin/data_aug.py"),
                is_stageable=True
            )


'''model_hpo = Transformation(
                "hpo",
                site="local",
                pfn=  os.path.join(os.getcwd(),"bin/model_hpo.py"),
                is_stageable=True
            )'''

'model_hpo = Transformation(\n                "hpo",\n                site="local",\n                pfn=  os.path.join(os.getcwd(),"bin/model_hpo.py"),\n                is_stageable=True\n            )'

In [6]:
tc = TransformationCatalog()
tc.add_transformations(augment_imgs)#, model_hpo)
tc.write()

In [ ]:
#preprocess_val_job = Job(preprocess_imgs)
#preprocess_val_job.add_inputs(*val_imgs)
#preprocess_val_job.add_outputs(*val_preprocessed_files)

#val_preprocessed_files   = create_preprocessed_filelist(val_filenames)
#test_preprocessed_files  = create_preprocessed_filelist(test_filenames)

#preprocess_test_job = Job(preprocess_imgs)
#preprocess_test_job.add_inputs(*test_imgs)
#preprocess_test_job.add_outputs(*test_preprocessed_files)

In [7]:
#Create workflow

wf = Workflow("mask_detection_workflow")

train_preprocessed_files = create_augmented_filelist(train_filenames,1)

# Add jobs to the workflow


# DATA EXPLORATION
# takes in all the annotationa files and creates plot with distribution of the classes


# DATA AUGMENTATION
# takes images and adds gaussian noise to them
preprocess_train_job = Job(augment_imgs)
preprocess_train_job.add_inputs(*train_imgs)
preprocess_train_job.add_outputs(*train_preprocessed_files)


preprocess_val_job = Job(preprocess_imgs)
preprocess_val_job.add_inputs(*val_imgs)
preprocess_val_job.add_outputs(*val_preprocessed_files)


preprocess_test_job = Job(preprocess_imgs)
preprocess_test_job.add_inputs(*test_imgs)
preprocess_test_job.add_outputs(*test_preprocessed_files)


# HYPERPARAMETER OPTIMIZATION
#hpo_job = Job(model_hpo)
#hpo_job.add_inputs(*train_preprocessed_files, *val_preprocessed_files)
#hpo_job.add_inputs(hpo_results)

# MODEL TRAINING and EVALUATION


# INFERENCE
# takes images of our labmates and classifies them



In [8]:
# add jobs to workflow
wf.add_jobs(
    preprocess_train_job,
)

# run workflow
try:
    wf.plan(submit=True)
    wf.wait()
    wf.statistics()
except PegasusClientError as e:
    print(e.output)

2021.02.18 15:59:55.383 UTC:
2021.02.18 15:59:55.388 UTC:   -----------------------------------------------------------------------
2021.02.18 15:59:55.394 UTC:   File for submitting this DAG to HTCondor           : mask_detection_workflow-0.dag.condor.sub
2021.02.18 15:59:55.399 UTC:   Log of DAGMan debugging messages                 : mask_detection_workflow-0.dag.dagman.out
2021.02.18 15:59:55.404 UTC:   Log of HTCondor library output                     : mask_detection_workflow-0.dag.lib.out
2021.02.18 15:59:55.409 UTC:   Log of HTCondor library error messages             : mask_detection_workflow-0.dag.lib.err
2021.02.18 15:59:55.414 UTC:   Log of the life of condor_dagman itself          : mask_detection_workflow-0.dag.dagman.log
2021.02.18 15:59:55.420 UTC:
2021.02.18 15:59:55.425 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with:
2021.02.18 15:59:55.435 UTC:   -----------------------------------------------------------------------
2021.02.18 15:59:

[####################################] 100.0% ..Success (Completed: 9, Queued: 0, Running: 0, Failed: 0)
